In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import os
from zipfile import ZipFile

# Step 1: Unzip dataset if needed
dataset_zip_path = r"/content/Dataset.zip"
extracted_dir = r"/content/Dataset"

if not os.path.exists(extracted_dir):
    with ZipFile(dataset_zip_path, 'r') as zip_ref:
        zip_ref.extractall(extracted_dir)
        print(f"✅ Dataset unzipped to {extracted_dir}")
else:
    print(f"✅ Dataset already extracted at {extracted_dir}")

# Step 2: Path to training dataset
train_dir = os.path.join(extracted_dir, 'Dataset', 'Test')

# Feature extractor using DenseNet121
class DenseNetFeatureExtractor(nn.Module):
    def __init__(self, output_dim=1024):
        super(DenseNetFeatureExtractor, self).__init__()
        densenet = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
        self.features = densenet.features
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(1024, output_dim)  # Ensure 1024 output

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = self.flatten(x)
        return self.fc(x)  # Ensures correct output size

# CrossViT Transformer block
class CrossViT(nn.Module):
    def __init__(self, input_dim=1024, hidden_dim=512, num_heads=8, num_layers=3):
        super(CrossViT, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=input_dim, nhead=num_heads, dim_feedforward=hidden_dim
        )
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)

    def forward(self, tokens):
        return self.transformer_encoder(tokens)

# Full DeepFake Detector Model
class DeepFakeDetector(nn.Module):
    def __init__(self):
        super(DeepFakeDetector, self).__init__()
        self.dense_feature_extractor = DenseNetFeatureExtractor()
        self.cross_vit = CrossViT(input_dim=1024)
        self.classifier = nn.Linear(1024, 2)

    def forward(self, x):
        features = self.dense_feature_extractor(x)
        features = features.unsqueeze(1)
        encoded_features = self.cross_vit(features)
        return self.classifier(encoded_features[:, 0, :])

# Initialize model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DeepFakeDetector().to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

# Step 3: Load dataset with augmentation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    total_loss, correct, total = 0, 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        predicted = torch.argmax(outputs, dim=1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    accuracy = 100 * correct / total
    scheduler.step(total_loss)  # Adjust learning rate if loss stagnates
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}, Accuracy: {accuracy:.2f}%")

# Save trained model
torch.save(model.state_dict(), "deepfake_model.pth")
print("✅ Model training complete! Saved as deepfake_model.pth")


✅ Dataset already extracted at /content/Dataset


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch [1/20], Loss: 4.2004, Accuracy: 46.00%
Epoch [2/20], Loss: 3.9918, Accuracy: 54.00%
Epoch [3/20], Loss: 3.3827, Accuracy: 79.00%
Epoch [4/20], Loss: 0.8274, Accuracy: 89.00%
Epoch [5/20], Loss: 0.7948, Accuracy: 91.00%
Epoch [6/20], Loss: 0.7400, Accuracy: 92.00%
Epoch [7/20], Loss: 0.6159, Accuracy: 94.00%
Epoch [8/20], Loss: 0.6402, Accuracy: 97.00%
Epoch [9/20], Loss: 0.8129, Accuracy: 95.00%
Epoch [10/20], Loss: 0.2900, Accuracy: 96.00%
Epoch [11/20], Loss: 0.2313, Accuracy: 98.00%
Epoch [12/20], Loss: 0.0779, Accuracy: 100.00%
Epoch [13/20], Loss: 0.3254, Accuracy: 96.00%
Epoch [14/20], Loss: 0.3430, Accuracy: 97.00%
Epoch [15/20], Loss: 0.7164, Accuracy: 97.00%
Epoch [16/20], Loss: 0.6958, Accuracy: 98.00%
Epoch [17/20], Loss: 0.4856, Accuracy: 95.00%
Epoch [18/20], Loss: 0.3044, Accuracy: 97.00%
Epoch [19/20], Loss: 0.1998, Accuracy: 99.00%
Epoch [20/20], Loss: 0.8179, Accuracy: 97.00%
✅ Model training complete! Saved as deepfake_model.pth


In [ ]:
import torch
import os
import zipfile
from PIL import Image
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from torch import nn
from torchvision import models

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Ensure dataset is extracted before use
zip_path = "/content/Test.zip"
extract_to = "/content/Test"

if os.path.exists(zip_path) and zipfile.is_zipfile(zip_path):
    print("🔄 Extracting testdata.zip...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print("✅ Extraction complete!")

# Update dataset directory after extraction
extracted_test_dir = extract_to

# Check dataset presence
if not os.path.exists(extracted_test_dir) or not os.listdir(extracted_test_dir):
    print(f"❌ No images found in {extracted_test_dir}. Please verify the dataset.")
    exit()

# Dataset class for loading images from subdirectories
class TestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.data = []

        # Scan subdirectories
        for subdir, _, files in os.walk(root_dir):
            for img_name in files:
                if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img_path = os.path.join(subdir, img_name)

                    # Assign label based on subdirectory or filename
                    label = 0 if "real" in subdir.lower() or "real" in img_name.lower() else 1
                    self.data.append((img_path, label))

        print(f"✅ Found {len(self.data)*3} images in the dataset.")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

# Feature extractor using DenseNet
class DenseNetFeatureExtractor(nn.Module):
    def __init__(self):
        super(DenseNetFeatureExtractor, self).__init__()
        densenet = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT)
        self.features = densenet.features
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = self.flatten(x)  # Shape: (batch_size, 1024)
        return x

# Transformer-based classifier (CrossViT)
class CrossViT(nn.Module):
    def __init__(self, input_dim=1024, hidden_dim=512, num_heads=8, num_layers=2):
        super(CrossViT, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=input_dim, nhead=num_heads, dim_feedforward=hidden_dim, batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)

    def forward(self, tokens):
        return self.transformer_encoder(tokens)

# DeepFake Detection Model
class DeepFakeDetector(nn.Module):
    def __init__(self):
        super(DeepFakeDetector, self).__init__()
        self.dense_feature_extractor = DenseNetFeatureExtractor()
        self.cross_vit = CrossViT(input_dim=1024)
        self.classifier = nn.Linear(1024, 2)

    def forward(self, x):
        features = self.dense_feature_extractor(x)
        features = features.unsqueeze(1)  # Shape: (batch_size, 1, 1024)
        encoded_features = self.cross_vit(features)
        output = self.classifier(encoded_features[:, 0, :])
        return output

# Load model and weights
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DeepFakeDetector().to(device)

model_path = "/content/deepfake_model.pth"
if os.path.exists(model_path):
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint, strict=False)
    model.eval()
    print("✅ Model loaded successfully!")
else:
    print(f"❌ Model file not found at {model_path}")
    exit()

# Load dataset
test_dataset = TestDataset(root_dir=extracted_test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Model evaluation
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        predicted = torch.argmax(outputs, dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

if total > 0:
    accuracy = 100 * correct / total
    print(f"✅ Model Test Accuracy: {accuracy:.2f}%")
else:
    print("❌ No data to evaluate.")


🔄 Extracting testdata.zip...
✅ Extraction complete!


<ipython-input-5-6dd2da831e31>:112: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path, map_location=device)


✅ Model loaded successfully!
✅ Found 300 images in the dataset.
✅ Model Test Accuracy: 52.00%


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import os
from zipfile import ZipFile

# Step 1: Unzip dataset if needed
dataset_zip_path = r"/content/Dataset.zip"
extracted_dir = r"/content/Dataset"

if not os.path.exists(extracted_dir):
    with ZipFile(dataset_zip_path, 'r') as zip_ref:
        zip_ref.extractall(extracted_dir)
        print(f"✅ Dataset unzipped to {extracted_dir}")
else:
    print(f"✅ Dataset already extracted at {extracted_dir}")

# Step 2: Path to training dataset
train_dir = os.path.join(extracted_dir, 'Dataset', 'Test')

# Feature extractor using DenseNet121
class DenseNetFeatureExtractor(nn.Module):
    def __init__(self, output_dim=1024):
        super(DenseNetFeatureExtractor, self).__init__()
        densenet = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
        self.features = densenet.features
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(1024, output_dim)  # Ensure 1024 output

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = self.flatten(x)
        return self.fc(x)  # Ensures correct output size

# CrossViT Transformer block
class CrossViT(nn.Module):
    def __init__(self, input_dim=1024, hidden_dim=512, num_heads=8, num_layers=3):
        super(CrossViT, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=input_dim, nhead=num_heads, dim_feedforward=hidden_dim
        )
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)

    def forward(self, tokens):
        return self.transformer_encoder(tokens)

# Full DeepFake Detector Model
class DeepFakeDetector(nn.Module):
    def __init__(self):
        super(DeepFakeDetector, self).__init__()
        self.dense_feature_extractor = DenseNetFeatureExtractor()
        self.cross_vit = CrossViT(input_dim=1024)
        self.classifier = nn.Linear(1024, 2)

    def forward(self, x):
        features = self.dense_feature_extractor(x)
        features = features.unsqueeze(1)
        encoded_features = self.cross_vit(features)
        return self.classifier(encoded_features[:, 0, :])

# Initialize model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DeepFakeDetector().to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

# Step 3: Load dataset with augmentation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    total_loss, correct, total = 0, 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        predicted = torch.argmax(outputs, dim=1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    accuracy = 100 * correct / total
    scheduler.step(total_loss)  # Adjust learning rate if loss stagnates
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}, Accuracy: {accuracy:.2f}%")

# Save trained model
torch.save(model.state_dict(), "deepfake_model.pth")
print("✅ Model training complete! Saved as deepfake_model.pth")


✅ Dataset already extracted at /content/Dataset
Epoch [1/20], Loss: 5.0230, Accuracy: 46.00%
Epoch [2/20], Loss: 2.3059, Accuracy: 63.00%
Epoch [3/20], Loss: 1.6708, Accuracy: 81.00%
Epoch [4/20], Loss: 1.1032, Accuracy: 87.00%
Epoch [5/20], Loss: 0.6454, Accuracy: 92.00%
Epoch [6/20], Loss: 0.9049, Accuracy: 95.00%
Epoch [7/20], Loss: 0.3585, Accuracy: 95.00%
Epoch [8/20], Loss: 0.3598, Accuracy: 94.00%
Epoch [9/20], Loss: 1.1648, Accuracy: 95.00%
Epoch [10/20], Loss: 1.9244, Accuracy: 98.00%
Epoch [11/20], Loss: 0.6116, Accuracy: 99.00%
Epoch [12/20], Loss: 0.4886, Accuracy: 95.00%
Epoch [13/20], Loss: 0.2577, Accuracy: 97.00%
Epoch [14/20], Loss: 0.4149, Accuracy: 98.00%
Epoch [15/20], Loss: 0.2724, Accuracy: 97.00%
Epoch [16/20], Loss: 0.2229, Accuracy: 97.00%
Epoch [17/20], Loss: 0.2217, Accuracy: 98.00%
Epoch [18/20], Loss: 0.3256, Accuracy: 97.00%
Epoch [19/20], Loss: 0.8178, Accuracy: 97.00%
Epoch [20/20], Loss: 0.8824, Accuracy: 97.00%
✅ Model training complete! Saved as deepf

In [ ]:
import torch
import os
import zipfile
from PIL import Image
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from torch import nn
from torchvision import models

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Ensure dataset is extracted before use
zip_path = "/content/Test.zip"
extract_to = "/content/Test"

if os.path.exists(zip_path) and zipfile.is_zipfile(zip_path):
    print("🔄 Extracting testdata.zip...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print("✅ Extraction complete!")

# Update dataset directory after extraction
extracted_test_dir = extract_to

# Check dataset presence
if not os.path.exists(extracted_test_dir) or not os.listdir(extracted_test_dir):
    print(f"❌ No images found in {extracted_test_dir}. Please verify the dataset.")
    exit()

# Dataset class for loading images from subdirectories
class TestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.data = []

        # Scan subdirectories
        for subdir, _, files in os.walk(root_dir):
            for img_name in files:
                if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img_path = os.path.join(subdir, img_name)

                    # Assign label based on subdirectory or filename
                    label = 0 if "real" in subdir.lower() or "real" in img_name.lower() else 1
                    self.data.append((img_path, label))

        print(f"✅ Found {len(self.data)*3} images in the dataset.")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

# Feature extractor using DenseNet
class DenseNetFeatureExtractor(nn.Module):
    def __init__(self):
        super(DenseNetFeatureExtractor, self).__init__()
        densenet = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT)
        self.features = densenet.features
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = self.flatten(x)  # Shape: (batch_size, 1024)
        return x

# Transformer-based classifier (CrossViT)
class CrossViT(nn.Module):
    def __init__(self, input_dim=1024, hidden_dim=512, num_heads=8, num_layers=2):
        super(CrossViT, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=input_dim, nhead=num_heads, dim_feedforward=hidden_dim, batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)

    def forward(self, tokens):
        return self.transformer_encoder(tokens)

# DeepFake Detection Model
class DeepFakeDetector(nn.Module):
    def __init__(self):
        super(DeepFakeDetector, self).__init__()
        self.dense_feature_extractor = DenseNetFeatureExtractor()
        self.cross_vit = CrossViT(input_dim=1024)
        self.classifier = nn.Linear(1024, 2)

    def forward(self, x):
        features = self.dense_feature_extractor(x)
        features = features.unsqueeze(1)  # Shape: (batch_size, 1, 1024)
        encoded_features = self.cross_vit(features)
        output = self.classifier(encoded_features[:, 0, :])
        return output

# Load model and weights
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DeepFakeDetector().to(device)

model_path = "/content/deepfake_model.pth"
if os.path.exists(model_path):
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint, strict=False)
    model.eval()
    print("✅ Model loaded successfully!")
else:
    print(f"❌ Model file not found at {model_path}")
    exit()

# Load dataset
test_dataset = TestDataset(root_dir=extracted_test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Model evaluation
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        predicted = torch.argmax(outputs, dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

if total > 0:
    accuracy = 100 * correct / total
    print(f"✅ Model Test Accuracy: {accuracy:.2f}%")
else:
    print("❌ No data to evaluate.")


🔄 Extracting testdata.zip...
✅ Extraction complete!


<ipython-input-11-6dd2da831e31>:112: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path, map_location=device)


✅ Model loaded successfully!
✅ Found 300 images in the dataset.
✅ Model Test Accuracy: 40.00%


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import os
from zipfile import ZipFile

# Step 1: Unzip dataset if needed
dataset_zip_path = r"/content/Dataset.zip"
extracted_dir = r"/content/Dataset"

if not os.path.exists(extracted_dir):
    with ZipFile(dataset_zip_path, 'r') as zip_ref:
        zip_ref.extractall(extracted_dir)
        print(f"✅ Dataset unzipped to {extracted_dir}")
else:
    print(f"✅ Dataset already extracted at {extracted_dir}")

# Step 2: Path to training dataset
train_dir = os.path.join(extracted_dir, 'Dataset', 'Test')

# Feature extractor using DenseNet121
class DenseNetFeatureExtractor(nn.Module):
    def __init__(self, output_dim=1024):
        super(DenseNetFeatureExtractor, self).__init__()
        densenet = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
        self.features = densenet.features
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(1024, output_dim)  # Ensure 1024 output

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = self.flatten(x)
        return self.fc(x)  # Ensures correct output size

# CrossViT Transformer block
class CrossViT(nn.Module):
    def __init__(self, input_dim=1024, hidden_dim=512, num_heads=8, num_layers=3):
        super(CrossViT, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=input_dim, nhead=num_heads, dim_feedforward=hidden_dim
        )
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)

    def forward(self, tokens):
        return self.transformer_encoder(tokens)

# Full DeepFake Detector Model
class DeepFakeDetector(nn.Module):
    def __init__(self):
        super(DeepFakeDetector, self).__init__()
        self.dense_feature_extractor = DenseNetFeatureExtractor()
        self.cross_vit = CrossViT(input_dim=1024)
        self.classifier = nn.Linear(1024, 2)

    def forward(self, x):
        features = self.dense_feature_extractor(x)
        features = features.unsqueeze(1)
        encoded_features = self.cross_vit(features)
        return self.classifier(encoded_features[:, 0, :])

# Initialize model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DeepFakeDetector().to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

# Step 3: Load dataset with augmentation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    total_loss, correct, total = 0, 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        predicted = torch.argmax(outputs, dim=1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    accuracy = 100 * correct / total
    scheduler.step(total_loss)  # Adjust learning rate if loss stagnates
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}, Accuracy: {accuracy:.2f}%")

# Save trained model
torch.save(model.state_dict(), "deepfake_model.pth")
print("✅ Model training complete! Saved as deepfake_model.pth")


✅ Dataset unzipped to /content/Dataset


Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 122MB/s]
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch [1/20], Loss: 5.6736, Accuracy: 47.00%
Epoch [2/20], Loss: 2.1181, Accuracy: 70.00%
Epoch [3/20], Loss: 1.9107, Accuracy: 83.00%
Epoch [4/20], Loss: 0.7749, Accuracy: 90.00%
Epoch [5/20], Loss: 0.8125, Accuracy: 88.00%
Epoch [6/20], Loss: 0.4200, Accuracy: 95.00%
Epoch [7/20], Loss: 0.4434, Accuracy: 96.00%
Epoch [8/20], Loss: 0.3507, Accuracy: 95.00%
Epoch [9/20], Loss: 0.1244, Accuracy: 98.00%
Epoch [10/20], Loss: 0.0702, Accuracy: 100.00%
Epoch [11/20], Loss: 0.2799, Accuracy: 96.00%
Epoch [12/20], Loss: 0.0795, Accuracy: 99.00%
Epoch [13/20], Loss: 0.9246, Accuracy: 97.00%
Epoch [14/20], Loss: 0.0867, Accuracy: 99.00%
Epoch [15/20], Loss: 0.7494, Accuracy: 95.00%
Epoch [16/20], Loss: 0.4213, Accuracy: 93.00%
Epoch [17/20], Loss: 0.6749, Accuracy: 94.00%
Epoch [18/20], Loss: 2.2996, Accuracy: 97.00%
Epoch [19/20], Loss: 0.1280, Accuracy: 100.00%
Epoch [20/20], Loss: 0.0666, Accuracy: 100.00%
✅ Model training complete! Saved as deepfake_model.pth


In [ ]:
import torch
import os
import zipfile
from PIL import Image
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from torch import nn
from torchvision import models

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Ensure dataset is extracted before use
zip_path = "/content/Test.zip"
extract_to = "/content/Test"

if os.path.exists(zip_path) and zipfile.is_zipfile(zip_path):
    print("🔄 Extracting testdata.zip...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print("✅ Extraction complete!")

# Update dataset directory after extraction
extracted_test_dir = extract_to

# Check dataset presence
if not os.path.exists(extracted_test_dir) or not os.listdir(extracted_test_dir):
    print(f"❌ No images found in {extracted_test_dir}. Please verify the dataset.")
    exit()

# Dataset class for loading images from subdirectories
class TestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.data = []

        # Scan subdirectories
        for subdir, _, files in os.walk(root_dir):
            for img_name in files:
                if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img_path = os.path.join(subdir, img_name)

                    # Assign label based on subdirectory or filename
                    label = 0 if "real" in subdir.lower() or "real" in img_name.lower() else 1
                    self.data.append((img_path, label))

        print(f"✅ Found {len(self.data)*3} images in the dataset.")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

# Feature extractor using DenseNet
class DenseNetFeatureExtractor(nn.Module):
    def __init__(self):
        super(DenseNetFeatureExtractor, self).__init__()
        densenet = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT)
        self.features = densenet.features
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = self.flatten(x)  # Shape: (batch_size, 1024)
        return x

# Transformer-based classifier (CrossViT)
class CrossViT(nn.Module):
    def __init__(self, input_dim=1024, hidden_dim=512, num_heads=8, num_layers=2):
        super(CrossViT, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=input_dim, nhead=num_heads, dim_feedforward=hidden_dim, batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)

    def forward(self, tokens):
        return self.transformer_encoder(tokens)

# DeepFake Detection Model
class DeepFakeDetector(nn.Module):
    def __init__(self):
        super(DeepFakeDetector, self).__init__()
        self.dense_feature_extractor = DenseNetFeatureExtractor()
        self.cross_vit = CrossViT(input_dim=1024)
        self.classifier = nn.Linear(1024, 2)

    def forward(self, x):
        features = self.dense_feature_extractor(x)
        features = features.unsqueeze(1)  # Shape: (batch_size, 1, 1024)
        encoded_features = self.cross_vit(features)
        output = self.classifier(encoded_features[:, 0, :])
        return output

# Load model and weights
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DeepFakeDetector().to(device)

model_path = "/content/deepfake_model.pth"
if os.path.exists(model_path):
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint, strict=False)
    model.eval()
    print("✅ Model loaded successfully!")
else:
    print(f"❌ Model file not found at {model_path}")
    exit()

# Load dataset
test_dataset = TestDataset(root_dir=extracted_test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Model evaluation
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        predicted = torch.argmax(outputs, dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

if total > 0:
    accuracy = 100 * correct / total
    print(f"✅ Model Test Accuracy: {accuracy:.2f}%")
else:
    print("❌ No data to evaluate.")


🔄 Extracting testdata.zip...
✅ Extraction complete!
✅ Model loaded successfully!
✅ Found 300 images in the dataset.
✅ Model Test Accuracy: 52.00%


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import os
from zipfile import ZipFile

# Step 1: Unzip dataset if needed
dataset_zip_path = r"/content/Dataset.zip"
extracted_dir = r"/content/Dataset"

if not os.path.exists(extracted_dir):
    with ZipFile(dataset_zip_path, 'r') as zip_ref:
        zip_ref.extractall(extracted_dir)
        print(f"✅ Dataset unzipped to {extracted_dir}")
else:
    print(f"✅ Dataset already extracted at {extracted_dir}")

# Step 2: Path to training dataset
train_dir = os.path.join(extracted_dir, 'Dataset', 'Test')

# Feature extractor using DenseNet121
class DenseNetFeatureExtractor(nn.Module):
    def __init__(self, output_dim=1024):
        super(DenseNetFeatureExtractor, self).__init__()
        densenet = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
        self.features = densenet.features
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(1024, output_dim)  # Ensure 1024 output

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = self.flatten(x)
        return self.fc(x)  # Ensures correct output size

# CrossViT Transformer block
class CrossViT(nn.Module):
    def __init__(self, input_dim=1024, hidden_dim=512, num_heads=8, num_layers=3):
        super(CrossViT, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=input_dim, nhead=num_heads, dim_feedforward=hidden_dim
        )
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)

    def forward(self, tokens):
        return self.transformer_encoder(tokens)

# Full DeepFake Detector Model
class DeepFakeDetector(nn.Module):
    def __init__(self):
        super(DeepFakeDetector, self).__init__()
        self.dense_feature_extractor = DenseNetFeatureExtractor()
        self.cross_vit = CrossViT(input_dim=1024)
        self.classifier = nn.Linear(1024, 2)

    def forward(self, x):
        features = self.dense_feature_extractor(x)
        features = features.unsqueeze(1)
        encoded_features = self.cross_vit(features)
        return self.classifier(encoded_features[:, 0, :])

# Initialize model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DeepFakeDetector().to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

# Step 3: Load dataset with augmentation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    total_loss, correct, total = 0, 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        predicted = torch.argmax(outputs, dim=1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    accuracy = 100 * correct / total
    scheduler.step(total_loss)  # Adjust learning rate if loss stagnates
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}, Accuracy: {accuracy:.2f}%")

# Save trained model
torch.save(model.state_dict(), "deepfake_model.pth")
print("✅ Model training complete! Saved as deepfake_model.pth")


✅ Dataset already extracted at /content/Dataset
Epoch [1/20], Loss: 5.1228, Accuracy: 45.00%
Epoch [2/20], Loss: 2.5593, Accuracy: 66.00%
Epoch [3/20], Loss: 1.3413, Accuracy: 92.00%
Epoch [4/20], Loss: 1.0495, Accuracy: 88.00%
Epoch [5/20], Loss: 1.7740, Accuracy: 91.00%
Epoch [6/20], Loss: 0.9721, Accuracy: 93.00%
Epoch [7/20], Loss: 0.6521, Accuracy: 93.00%
Epoch [8/20], Loss: 1.0574, Accuracy: 96.00%
Epoch [9/20], Loss: 0.3041, Accuracy: 97.00%
Epoch [10/20], Loss: 0.3734, Accuracy: 96.00%
Epoch [11/20], Loss: 0.4158, Accuracy: 97.00%
Epoch [12/20], Loss: 0.0883, Accuracy: 99.00%
Epoch [13/20], Loss: 0.1843, Accuracy: 97.00%
Epoch [14/20], Loss: 0.3190, Accuracy: 98.00%
Epoch [15/20], Loss: 0.2172, Accuracy: 99.00%
Epoch [16/20], Loss: 0.7477, Accuracy: 97.00%
Epoch [17/20], Loss: 0.1007, Accuracy: 99.00%
Epoch [18/20], Loss: 0.0331, Accuracy: 100.00%
Epoch [19/20], Loss: 0.4919, Accuracy: 97.00%
Epoch [20/20], Loss: 0.0677, Accuracy: 100.00%
✅ Model training complete! Saved as dee

In [ ]:
import torch
import os
import zipfile
from PIL import Image
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from torch import nn
from torchvision import models

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Ensure dataset is extracted before use
zip_path = "/content/Test.zip"
extract_to = "/content/Test"

if os.path.exists(zip_path) and zipfile.is_zipfile(zip_path):
    print("🔄 Extracting testdata.zip...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print("✅ Extraction complete!")

# Update dataset directory after extraction
extracted_test_dir = extract_to

# Check dataset presence
if not os.path.exists(extracted_test_dir) or not os.listdir(extracted_test_dir):
    print(f"❌ No images found in {extracted_test_dir}. Please verify the dataset.")
    exit()

# Dataset class for loading images from subdirectories
class TestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.data = []

        # Scan subdirectories
        for subdir, _, files in os.walk(root_dir):
            for img_name in files:
                if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img_path = os.path.join(subdir, img_name)

                    # Assign label based on subdirectory or filename
                    label = 0 if "real" in subdir.lower() or "real" in img_name.lower() else 1
                    self.data.append((img_path, label))

        print(f"✅ Found {len(self.data)*3} images in the dataset.")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

# Feature extractor using DenseNet
class DenseNetFeatureExtractor(nn.Module):
    def __init__(self):
        super(DenseNetFeatureExtractor, self).__init__()
        densenet = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT)
        self.features = densenet.features
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = self.flatten(x)  # Shape: (batch_size, 1024)
        return x

# Transformer-based classifier (CrossViT)
class CrossViT(nn.Module):
    def __init__(self, input_dim=1024, hidden_dim=512, num_heads=8, num_layers=2):
        super(CrossViT, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=input_dim, nhead=num_heads, dim_feedforward=hidden_dim, batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)

    def forward(self, tokens):
        return self.transformer_encoder(tokens)

# DeepFake Detection Model
class DeepFakeDetector(nn.Module):
    def __init__(self):
        super(DeepFakeDetector, self).__init__()
        self.dense_feature_extractor = DenseNetFeatureExtractor()
        self.cross_vit = CrossViT(input_dim=1024)
        self.classifier = nn.Linear(1024, 2)

    def forward(self, x):
        features = self.dense_feature_extractor(x)
        features = features.unsqueeze(1)  # Shape: (batch_size, 1, 1024)
        encoded_features = self.cross_vit(features)
        output = self.classifier(encoded_features[:, 0, :])
        return output

# Load model and weights
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DeepFakeDetector().to(device)

model_path = "/content/deepfake_model.pth"
if os.path.exists(model_path):
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint, strict=False)
    model.eval()
    print("✅ Model loaded successfully!")
else:
    print(f"❌ Model file not found at {model_path}")
    exit()

# Load dataset
test_dataset = TestDataset(root_dir=extracted_test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Model evaluation
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        predicted = torch.argmax(outputs, dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

if total > 0:
    accuracy = 100 * correct / total
    print(f"✅ Model Test Accuracy: {accuracy:.2f}%")
else:
    print("❌ No data to evaluate.")


🔄 Extracting testdata.zip...
✅ Extraction complete!
✅ Model loaded successfully!
✅ Found 300 images in the dataset.
✅ Model Test Accuracy: 49.00%


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import os
from zipfile import ZipFile

# Step 1: Unzip dataset if needed
dataset_zip_path = r"/content/Dataset.zip"
extracted_dir = r"/content/Dataset"

if not os.path.exists(extracted_dir):
    with ZipFile(dataset_zip_path, 'r') as zip_ref:
        zip_ref.extractall(extracted_dir)
        print(f"✅ Dataset unzipped to {extracted_dir}")
else:
    print(f"✅ Dataset already extracted at {extracted_dir}")

# Step 2: Path to training dataset
train_dir = os.path.join(extracted_dir, 'Dataset', 'Test')

# Feature extractor using DenseNet121
class DenseNetFeatureExtractor(nn.Module):
    def __init__(self, output_dim=1024):
        super(DenseNetFeatureExtractor, self).__init__()
        densenet = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
        self.features = densenet.features
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(1024, output_dim)  # Ensure 1024 output

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = self.flatten(x)
        return self.fc(x)  # Ensures correct output size

# CrossViT Transformer block
class CrossViT(nn.Module):
    def __init__(self, input_dim=1024, hidden_dim=512, num_heads=8, num_layers=3):
        super(CrossViT, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=input_dim, nhead=num_heads, dim_feedforward=hidden_dim
        )
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)

    def forward(self, tokens):
        return self.transformer_encoder(tokens)

# Full DeepFake Detector Model
class DeepFakeDetector(nn.Module):
    def __init__(self):
        super(DeepFakeDetector, self).__init__()
        self.dense_feature_extractor = DenseNetFeatureExtractor()
        self.cross_vit = CrossViT(input_dim=1024)
        self.classifier = nn.Linear(1024, 2)

    def forward(self, x):
        features = self.dense_feature_extractor(x)
        features = features.unsqueeze(1)
        encoded_features = self.cross_vit(features)
        return self.classifier(encoded_features[:, 0, :])

# Initialize model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DeepFakeDetector().to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

# Step 3: Load dataset with augmentation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    total_loss, correct, total = 0, 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        predicted = torch.argmax(outputs, dim=1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    accuracy = 100 * correct / total
    scheduler.step(total_loss)  # Adjust learning rate if loss stagnates
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}, Accuracy: {accuracy:.2f}%")

# Save trained model
torch.save(model.state_dict(), "deepfake_model.pth")
print("✅ Model training complete! Saved as deepfake_model.pth")


✅ Dataset already extracted at /content/Dataset
Epoch [1/20], Loss: 3.2772, Accuracy: 47.00%
Epoch [2/20], Loss: 2.2386, Accuracy: 70.00%
Epoch [3/20], Loss: 1.4540, Accuracy: 88.00%
Epoch [4/20], Loss: 1.6060, Accuracy: 78.00%
Epoch [5/20], Loss: 1.7391, Accuracy: 92.00%
Epoch [6/20], Loss: 1.4805, Accuracy: 92.00%
Epoch [7/20], Loss: 0.8450, Accuracy: 94.00%
Epoch [8/20], Loss: 1.7041, Accuracy: 86.00%
Epoch [9/20], Loss: 0.3731, Accuracy: 93.00%
Epoch [10/20], Loss: 0.5387, Accuracy: 95.00%
Epoch [11/20], Loss: 0.5266, Accuracy: 96.00%
Epoch [12/20], Loss: 0.6522, Accuracy: 96.00%
Epoch [13/20], Loss: 1.4063, Accuracy: 97.00%
Epoch [14/20], Loss: 0.8130, Accuracy: 94.00%
Epoch [15/20], Loss: 0.8431, Accuracy: 96.00%
Epoch [16/20], Loss: 0.2458, Accuracy: 99.00%
Epoch [17/20], Loss: 0.2106, Accuracy: 97.00%
Epoch [18/20], Loss: 0.6533, Accuracy: 96.00%
Epoch [19/20], Loss: 0.1953, Accuracy: 99.00%
Epoch [20/20], Loss: 0.0903, Accuracy: 99.00%
✅ Model training complete! Saved as deepf

In [ ]:
import torch
import os
import zipfile
from PIL import Image
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from torch import nn
from torchvision import models

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Ensure dataset is extracted before use
zip_path = "/content/Test.zip"
extract_to = "/content/Test"

if os.path.exists(zip_path) and zipfile.is_zipfile(zip_path):
    print("🔄 Extracting testdata.zip...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print("✅ Extraction complete!")

# Update dataset directory after extraction
extracted_test_dir = extract_to

# Check dataset presence
if not os.path.exists(extracted_test_dir) or not os.listdir(extracted_test_dir):
    print(f"❌ No images found in {extracted_test_dir}. Please verify the dataset.")
    exit()

# Dataset class for loading images from subdirectories
class TestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.data = []

        # Scan subdirectories
        for subdir, _, files in os.walk(root_dir):
            for img_name in files:
                if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img_path = os.path.join(subdir, img_name)

                    # Assign label based on subdirectory or filename
                    label = 0 if "real" in subdir.lower() or "real" in img_name.lower() else 1
                    self.data.append((img_path, label))

        print(f"✅ Found {len(self.data)*3} images in the dataset.")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

# Feature extractor using DenseNet
class DenseNetFeatureExtractor(nn.Module):
    def __init__(self):
        super(DenseNetFeatureExtractor, self).__init__()
        densenet = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT)
        self.features = densenet.features
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = self.flatten(x)  # Shape: (batch_size, 1024)
        return x

# Transformer-based classifier (CrossViT)
class CrossViT(nn.Module):
    def __init__(self, input_dim=1024, hidden_dim=512, num_heads=8, num_layers=2):
        super(CrossViT, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=input_dim, nhead=num_heads, dim_feedforward=hidden_dim, batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)

    def forward(self, tokens):
        return self.transformer_encoder(tokens)

# DeepFake Detection Model
class DeepFakeDetector(nn.Module):
    def __init__(self):
        super(DeepFakeDetector, self).__init__()
        self.dense_feature_extractor = DenseNetFeatureExtractor()
        self.cross_vit = CrossViT(input_dim=1024)
        self.classifier = nn.Linear(1024, 2)

    def forward(self, x):
        features = self.dense_feature_extractor(x)
        features = features.unsqueeze(1)  # Shape: (batch_size, 1, 1024)
        encoded_features = self.cross_vit(features)
        output = self.classifier(encoded_features[:, 0, :])
        return output

# Load model and weights
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DeepFakeDetector().to(device)

model_path = "/content/deepfake_model.pth"
if os.path.exists(model_path):
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint, strict=False)
    model.eval()
    print("✅ Model loaded successfully!")
else:
    print(f"❌ Model file not found at {model_path}")
    exit()

# Load dataset
test_dataset = TestDataset(root_dir=extracted_test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Model evaluation
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        predicted = torch.argmax(outputs, dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

if total > 0:
    accuracy = 100 * correct / total
    print(f"✅ Model Test Accuracy: {accuracy:.2f}%")
else:
    print("❌ No data to evaluate.")


🔄 Extracting testdata.zip...
✅ Extraction complete!
✅ Model loaded successfully!
✅ Found 300 images in the dataset.
✅ Model Test Accuracy: 41.00%


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import os
from zipfile import ZipFile

# Step 1: Unzip dataset if needed
dataset_zip_path = r"/content/Dataset.zip"
extracted_dir = r"/content/Dataset"

if not os.path.exists(extracted_dir):
    with ZipFile(dataset_zip_path, 'r') as zip_ref:
        zip_ref.extractall(extracted_dir)
        print(f"✅ Dataset unzipped to {extracted_dir}")
else:
    print(f"✅ Dataset already extracted at {extracted_dir}")

# Step 2: Path to training dataset
train_dir = os.path.join(extracted_dir, 'Dataset', 'Test')

# Feature extractor using DenseNet121
class DenseNetFeatureExtractor(nn.Module):
    def __init__(self, output_dim=1024):
        super(DenseNetFeatureExtractor, self).__init__()
        densenet = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
        self.features = densenet.features
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(1024, output_dim)  # Ensure 1024 output

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = self.flatten(x)
        return self.fc(x)  # Ensures correct output size

# CrossViT Transformer block
class CrossViT(nn.Module):
    def __init__(self, input_dim=1024, hidden_dim=512, num_heads=8, num_layers=3):
        super(CrossViT, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=input_dim, nhead=num_heads, dim_feedforward=hidden_dim
        )
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)

    def forward(self, tokens):
        return self.transformer_encoder(tokens)

# Full DeepFake Detector Model
class DeepFakeDetector(nn.Module):
    def __init__(self):
        super(DeepFakeDetector, self).__init__()
        self.dense_feature_extractor = DenseNetFeatureExtractor()
        self.cross_vit = CrossViT(input_dim=1024)
        self.classifier = nn.Linear(1024, 2)

    def forward(self, x):
        features = self.dense_feature_extractor(x)
        features = features.unsqueeze(1)
        encoded_features = self.cross_vit(features)
        return self.classifier(encoded_features[:, 0, :])

# Initialize model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DeepFakeDetector().to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

# Step 3: Load dataset with augmentation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    total_loss, correct, total = 0, 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        predicted = torch.argmax(outputs, dim=1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    accuracy = 100 * correct / total
    scheduler.step(total_loss)  # Adjust learning rate if loss stagnates
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}, Accuracy: {accuracy:.2f}%")

# Save trained model
torch.save(model.state_dict(), "deepfake_model.pth")
print("✅ Model training complete! Saved as deepfake_model.pth")


✅ Dataset unzipped to /content/Dataset


Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 149MB/s]
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch [1/20], Loss: 5.1791, Accuracy: 50.00%
Epoch [2/20], Loss: 3.5516, Accuracy: 72.00%
Epoch [3/20], Loss: 1.2588, Accuracy: 86.00%
Epoch [4/20], Loss: 1.7239, Accuracy: 90.00%
Epoch [5/20], Loss: 0.8750, Accuracy: 90.00%
Epoch [6/20], Loss: 0.7122, Accuracy: 92.00%
Epoch [7/20], Loss: 0.7061, Accuracy: 86.00%
Epoch [8/20], Loss: 0.4879, Accuracy: 92.00%
Epoch [9/20], Loss: 0.2014, Accuracy: 97.00%
Epoch [10/20], Loss: 0.2498, Accuracy: 97.00%
Epoch [11/20], Loss: 0.1330, Accuracy: 99.00%
Epoch [12/20], Loss: 0.3577, Accuracy: 98.00%
Epoch [13/20], Loss: 0.1025, Accuracy: 99.00%
Epoch [14/20], Loss: 0.1677, Accuracy: 98.00%
Epoch [15/20], Loss: 0.1076, Accuracy: 99.00%
Epoch [16/20], Loss: 0.2444, Accuracy: 96.00%
Epoch [17/20], Loss: 0.0911, Accuracy: 99.00%
Epoch [18/20], Loss: 0.0792, Accuracy: 99.00%
Epoch [19/20], Loss: 0.0259, Accuracy: 100.00%
Epoch [20/20], Loss: 0.0503, Accuracy: 100.00%
✅ Model training complete! Saved as deepfake_model.pth


In [ ]:
import torch
import os
import zipfile
from PIL import Image
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from torch import nn
from torchvision import models

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Ensure dataset is extracted before use
zip_path = "/content/Test.zip"
extract_to = "/content/Test"

if os.path.exists(zip_path) and zipfile.is_zipfile(zip_path):
    print("🔄 Extracting testdata.zip...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print("✅ Extraction complete!")

# Update dataset directory after extraction
extracted_test_dir = extract_to

# Check dataset presence
if not os.path.exists(extracted_test_dir) or not os.listdir(extracted_test_dir):
    print(f"❌ No images found in {extracted_test_dir}. Please verify the dataset.")
    exit()

# Dataset class for loading images from subdirectories
class TestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.data = []

        # Scan subdirectories
        for subdir, _, files in os.walk(root_dir):
            for img_name in files:
                if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img_path = os.path.join(subdir, img_name)

                    # Assign label based on subdirectory or filename
                    label = 0 if "real" in subdir.lower() or "real" in img_name.lower() else 1
                    self.data.append((img_path, label))

        print(f"✅ Found {len(self.data)*3} images in the dataset.")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

# Feature extractor using DenseNet
class DenseNetFeatureExtractor(nn.Module):
    def __init__(self):
        super(DenseNetFeatureExtractor, self).__init__()
        densenet = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT)
        self.features = densenet.features
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = self.flatten(x)  # Shape: (batch_size, 1024)
        return x

# Transformer-based classifier (CrossViT)
class CrossViT(nn.Module):
    def __init__(self, input_dim=1024, hidden_dim=512, num_heads=8, num_layers=2):
        super(CrossViT, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=input_dim, nhead=num_heads, dim_feedforward=hidden_dim, batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)

    def forward(self, tokens):
        return self.transformer_encoder(tokens)

# DeepFake Detection Model
class DeepFakeDetector(nn.Module):
    def __init__(self):
        super(DeepFakeDetector, self).__init__()
        self.dense_feature_extractor = DenseNetFeatureExtractor()
        self.cross_vit = CrossViT(input_dim=1024)
        self.classifier = nn.Linear(1024, 2)

    def forward(self, x):
        features = self.dense_feature_extractor(x)
        features = features.unsqueeze(1)  # Shape: (batch_size, 1, 1024)
        encoded_features = self.cross_vit(features)
        output = self.classifier(encoded_features[:, 0, :])
        return output

# Load model and weights
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DeepFakeDetector().to(device)

model_path = "/content/deepfake_model.pth"
if os.path.exists(model_path):
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint, strict=False)
    model.eval()
    print("✅ Model loaded successfully!")
else:
    print(f"❌ Model file not found at {model_path}")
    exit()

# Load dataset
test_dataset = TestDataset(root_dir=extracted_test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Model evaluation
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        predicted = torch.argmax(outputs, dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

if total > 0:
    accuracy = 100 * correct / total
    print(f"✅ Model Test Accuracy: {accuracy:.2f}%")
else:
    print("❌ No data to evaluate.")


🔄 Extracting testdata.zip...
✅ Extraction complete!
✅ Model loaded successfully!
✅ Found 300 images in the dataset.
✅ Model Test Accuracy: 48.00%


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import os
from zipfile import ZipFile

# Step 1: Unzip dataset if needed
dataset_zip_path = r"/content/Dataset.zip"
extracted_dir = r"/content/Dataset"

if not os.path.exists(extracted_dir):
    with ZipFile(dataset_zip_path, 'r') as zip_ref:
        zip_ref.extractall(extracted_dir)
        print(f"✅ Dataset unzipped to {extracted_dir}")
else:
    print(f"✅ Dataset already extracted at {extracted_dir}")

# Step 2: Path to training dataset
train_dir = os.path.join(extracted_dir, 'Dataset', 'Test')

# Feature extractor using DenseNet121
class DenseNetFeatureExtractor(nn.Module):
    def __init__(self, output_dim=1024):
        super(DenseNetFeatureExtractor, self).__init__()
        densenet = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
        self.features = densenet.features
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(1024, output_dim)  # Ensure 1024 output

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = self.flatten(x)
        return self.fc(x)  # Ensures correct output size

# CrossViT Transformer block
class CrossViT(nn.Module):
    def __init__(self, input_dim=1024, hidden_dim=512, num_heads=8, num_layers=3):
        super(CrossViT, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=input_dim, nhead=num_heads, dim_feedforward=hidden_dim
        )
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)

    def forward(self, tokens):
        return self.transformer_encoder(tokens)

# Full DeepFake Detector Model
class DeepFakeDetector(nn.Module):
    def __init__(self):
        super(DeepFakeDetector, self).__init__()
        self.dense_feature_extractor = DenseNetFeatureExtractor()
        self.cross_vit = CrossViT(input_dim=1024)
        self.classifier = nn.Linear(1024, 2)

    def forward(self, x):
        features = self.dense_feature_extractor(x)
        features = features.unsqueeze(1)
        encoded_features = self.cross_vit(features)
        return self.classifier(encoded_features[:, 0, :])

# Initialize model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DeepFakeDetector().to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

# Step 3: Load dataset with augmentation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    total_loss, correct, total = 0, 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        predicted = torch.argmax(outputs, dim=1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    accuracy = 100 * correct / total
    scheduler.step(total_loss)  # Adjust learning rate if loss stagnates
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}, Accuracy: {accuracy:.2f}%")

# Save trained model
torch.save(model.state_dict(), "deepfake_model.pth")
print("✅ Model training complete! Saved as deepfake_model.pth")


✅ Dataset unzipped to /content/Dataset


Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 87.2MB/s]
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch [1/20], Loss: 5.6522, Accuracy: 52.00%
Epoch [2/20], Loss: 2.0648, Accuracy: 72.00%
Epoch [3/20], Loss: 1.5949, Accuracy: 83.00%
Epoch [4/20], Loss: 2.0131, Accuracy: 78.00%
Epoch [5/20], Loss: 1.2279, Accuracy: 80.00%
Epoch [6/20], Loss: 0.8143, Accuracy: 92.00%
Epoch [7/20], Loss: 1.9641, Accuracy: 94.00%
Epoch [8/20], Loss: 0.7873, Accuracy: 93.00%
Epoch [9/20], Loss: 0.3379, Accuracy: 95.00%
Epoch [10/20], Loss: 0.3483, Accuracy: 98.00%
Epoch [11/20], Loss: 1.3683, Accuracy: 95.00%
Epoch [12/20], Loss: 2.2900, Accuracy: 97.00%
Epoch [13/20], Loss: 1.1145, Accuracy: 93.00%
Epoch [14/20], Loss: 1.6160, Accuracy: 96.00%
Epoch [15/20], Loss: 0.5955, Accuracy: 97.00%
Epoch [16/20], Loss: 0.4501, Accuracy: 95.00%
Epoch [17/20], Loss: 0.2039, Accuracy: 100.00%
Epoch [18/20], Loss: 0.2692, Accuracy: 99.00%
Epoch [19/20], Loss: 0.5547, Accuracy: 97.00%
Epoch [20/20], Loss: 0.1290, Accuracy: 98.00%
✅ Model training complete! Saved as deepfake_model.pth


In [ ]:
import torch
import os
import zipfile
from PIL import Image
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from torch import nn
from torchvision import models

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Ensure dataset is extracted before use
zip_path = "/content/Test.zip"
extract_to = "/content/Test"

if os.path.exists(zip_path) and zipfile.is_zipfile(zip_path):
    print("🔄 Extracting testdata.zip...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print("✅ Extraction complete!")

# Update dataset directory after extraction
extracted_test_dir = extract_to

# Check dataset presence
if not os.path.exists(extracted_test_dir) or not os.listdir(extracted_test_dir):
    print(f"❌ No images found in {extracted_test_dir}. Please verify the dataset.")
    exit()

# Dataset class for loading images from subdirectories
class TestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.data = []

        # Scan subdirectories
        for subdir, _, files in os.walk(root_dir):
            for img_name in files:
                if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img_path = os.path.join(subdir, img_name)

                    # Assign label based on subdirectory or filename
                    label = 0 if "real" in subdir.lower() or "real" in img_name.lower() else 1
                    self.data.append((img_path, label))

        print(f"✅ Found {len(self.data)*3} images in the dataset.")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

# Feature extractor using DenseNet
class DenseNetFeatureExtractor(nn.Module):
    def __init__(self):
        super(DenseNetFeatureExtractor, self).__init__()
        densenet = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT)
        self.features = densenet.features
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = self.flatten(x)  # Shape: (batch_size, 1024)
        return x

# Transformer-based classifier (CrossViT)
class CrossViT(nn.Module):
    def __init__(self, input_dim=1024, hidden_dim=512, num_heads=8, num_layers=2):
        super(CrossViT, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=input_dim, nhead=num_heads, dim_feedforward=hidden_dim, batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)

    def forward(self, tokens):
        return self.transformer_encoder(tokens)

# DeepFake Detection Model
class DeepFakeDetector(nn.Module):
    def __init__(self):
        super(DeepFakeDetector, self).__init__()
        self.dense_feature_extractor = DenseNetFeatureExtractor()
        self.cross_vit = CrossViT(input_dim=1024)
        self.classifier = nn.Linear(1024, 2)

    def forward(self, x):
        features = self.dense_feature_extractor(x)
        features = features.unsqueeze(1)  # Shape: (batch_size, 1, 1024)
        encoded_features = self.cross_vit(features)
        output = self.classifier(encoded_features[:, 0, :])
        return output

# Load model and weights
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DeepFakeDetector().to(device)

model_path = "/content/deepfake_model.pth"
if os.path.exists(model_path):
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint, strict=False)
    model.eval()
    print("✅ Model loaded successfully!")
else:
    print(f"❌ Model file not found at {model_path}")
    exit()

# Load dataset
test_dataset = TestDataset(root_dir=extracted_test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Model evaluation
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        predicted = torch.argmax(outputs, dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

if total > 0:
    accuracy = 100 * correct / total
    print(f"✅ Model Test Accuracy: {accuracy:.2f}%")
else:
    print("❌ No data to evaluate.")


🔄 Extracting testdata.zip...
✅ Extraction complete!
✅ Model loaded successfully!
✅ Found 300 images in the dataset.
✅ Model Test Accuracy: 46.00%


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import os
from zipfile import ZipFile

# Step 1: Unzip dataset if needed
dataset_zip_path = r"/content/Dataset.zip"
extracted_dir = r"/content/Dataset"

if not os.path.exists(extracted_dir):
    with ZipFile(dataset_zip_path, 'r') as zip_ref:
        zip_ref.extractall(extracted_dir)
        print(f" Dataset unzipped to {extracted_dir}")
else:
    print(f" Dataset already extracted at {extracted_dir}")

# Step 2: Path to training dataset
train_dir = os.path.join(extracted_dir, 'Dataset', 'Test')

# Feature extractor using DenseNet121
class DenseNetFeatureExtractor(nn.Module):
    def __init__(self, output_dim=1024):
        super(DenseNetFeatureExtractor, self).__init__()
        densenet = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
        self.features = densenet.features
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(1024, output_dim)  # Ensure 1024 output

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = self.flatten(x)
        return self.fc(x)  # Ensures correct output size

# CrossViT Transformer block
class CrossViT(nn.Module):
    def __init__(self, input_dim=1024, hidden_dim=512, num_heads=8, num_layers=3):
        super(CrossViT, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=input_dim, nhead=num_heads, dim_feedforward=hidden_dim
        )
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)

    def forward(self, tokens):
        return self.transformer_encoder(tokens)

# Full DeepFake Detector Model
class DeepFakeDetector(nn.Module):
    def __init__(self):
        super(DeepFakeDetector, self).__init__()
        self.dense_feature_extractor = DenseNetFeatureExtractor()
        self.cross_vit = CrossViT(input_dim=1024)
        self.classifier = nn.Linear(1024, 2)

    def forward(self, x):
        features = self.dense_feature_extractor(x)
        features = features.unsqueeze(1)
        encoded_features = self.cross_vit(features)
        return self.classifier(encoded_features[:, 0, :])

# Initialize model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DeepFakeDetector().to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

# Step 3: Load dataset with augmentation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    total_loss, correct, total = 0, 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        predicted = torch.argmax(outputs, dim=1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    accuracy = 100 * correct / total
    scheduler.step(total_loss)  # Adjust learning rate if loss stagnates
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}, Accuracy: {accuracy:.2f}%")

# Save trained model
torch.save(model.state_dict(), "deepfake_model.pth")
print(" Model training complete! Saved as deepfake_model.pth")


 Dataset unzipped to /content/Dataset


Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 123MB/s]
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch [1/20], Loss: 4.3142, Accuracy: 52.00%
Epoch [2/20], Loss: 4.6982, Accuracy: 54.00%
Epoch [3/20], Loss: 1.6568, Accuracy: 77.00%
Epoch [4/20], Loss: 1.4861, Accuracy: 77.00%
Epoch [5/20], Loss: 0.6410, Accuracy: 95.00%
Epoch [6/20], Loss: 0.6949, Accuracy: 89.00%
Epoch [7/20], Loss: 1.9593, Accuracy: 93.00%
Epoch [8/20], Loss: 3.3368, Accuracy: 94.00%
Epoch [9/20], Loss: 0.5018, Accuracy: 96.00%
Epoch [10/20], Loss: 0.3327, Accuracy: 97.00%
Epoch [11/20], Loss: 1.1894, Accuracy: 88.00%
Epoch [12/20], Loss: 0.3743, Accuracy: 96.00%
Epoch [13/20], Loss: 0.2682, Accuracy: 97.00%
Epoch [14/20], Loss: 0.4491, Accuracy: 96.00%
Epoch [15/20], Loss: 0.9188, Accuracy: 98.00%
Epoch [16/20], Loss: 0.8099, Accuracy: 98.00%
Epoch [17/20], Loss: 0.6035, Accuracy: 96.00%
Epoch [18/20], Loss: 0.1353, Accuracy: 100.00%
Epoch [19/20], Loss: 0.4153, Accuracy: 98.00%
Epoch [20/20], Loss: 0.1801, Accuracy: 100.00%
 Model training complete! Saved as deepfake_model.pth


In [ ]:
import torch
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Directory to save feature images
feature_output_dir = "/content/densenet_features1"
os.makedirs(feature_output_dir, exist_ok=True)

# Path to test dataset (modify if needed)
test_dir = "/content/Dataset_f/Test"

# Define transformations
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load test dataset
test_dataset = datasets.ImageFolder(root=test_dir, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load trained model (ensure the model is defined and loaded properly)
model_path = "/content/deepfake_model.pth"
if os.path.exists(model_path):
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint, strict=False)
    model.eval()
    print("✅ Model loaded successfully!")
else:
    print(f"❌ Model file not found at {model_path}")
    exit()

# Model evaluation with F1-score calculation and feature extraction
correct = 0
total = 0
all_labels = []
all_predictions = []

with torch.no_grad():
    for batch_idx, (images, labels) in enumerate(test_loader):
        images, labels = images.to(device), labels.to(device)
        features = model.dense_feature_extractor(images)  # Extract DenseNet features

        # Save each feature map as an image
        for i in range(features.shape[0]):  # Iterate over batch
            feature_map = features[i].cpu().numpy()  # Convert to NumPy
            feature_map = np.mean(feature_map, axis=0)  # Average over channels

            # Normalize to 0-1 range
            feature_map = (feature_map - feature_map.min()) / (feature_map.max() - feature_map.min() + 1e-8)

            # Save feature image
            plt.imsave(os.path.join(feature_output_dir, f"feature_{batch_idx}_{i}.png"), feature_map, cmap='viridis')

        outputs = model(images)
        predicted = torch.argmax(outputs, dim=1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Store predictions and labels for F1-score calculation
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

if total > 0:
    accuracy = 100 * correct / total
    f1 = f1_score(all_labels, all_predictions, average='weighted')  # Weighted F1-score

    print(f" Model Test Accuracy: {accuracy:.2f}%")
    print(f" Model F1-Score: {f1:.4f}")
    print(f"Feature maps saved in: {feature_output_dir}")
else:
    print(" No data to evaluate.")


FileNotFoundError: [Errno 2] No such file or directory: '/content/Dataset_f/Test'

In [ ]:
import torch
import os
import zipfile
from PIL import Image
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from torch import nn
from torchvision import models

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Ensure dataset is extracted before use
zip_path = "/content/Dataset_f.zip"
extract_to = "/content/Dataset_f"

if os.path.exists(zip_path) and zipfile.is_zipfile(zip_path):
    print("🔄 Extracting testdata.zip...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print("✅ Extraction complete!")

# Update dataset directory after extraction
extracted_test_dir = extract_to

# Check dataset presence
if not os.path.exists(extracted_test_dir) or not os.listdir(extracted_test_dir):
    print(f"❌ No images found in {extracted_test_dir}. Please verify the dataset.")
    exit()

# Dataset class for loading images from subdirectories
class TestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.data = []

        # Scan subdirectories
        for subdir, _, files in os.walk(root_dir):
            for img_name in files:
                if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img_path = os.path.join(subdir, img_name)

                    # Assign label based on subdirectory or filename
                    label = 0 if "real" in subdir.lower() or "real" in img_name.lower() else 1
                    self.data.append((img_path, label))

        print(f"✅ Found {len(self.data)*3} images in the dataset.")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

# Feature extractor using DenseNet
class DenseNetFeatureExtractor(nn.Module):
    def __init__(self):
        super(DenseNetFeatureExtractor, self).__init__()
        densenet = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT)
        self.features = densenet.features
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = self.flatten(x)  # Shape: (batch_size, 1024)
        return x

# Transformer-based classifier (CrossViT)
class CrossViT(nn.Module):
    def __init__(self, input_dim=1024, hidden_dim=512, num_heads=8, num_layers=2):
        super(CrossViT, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=input_dim, nhead=num_heads, dim_feedforward=hidden_dim, batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)

    def forward(self, tokens):
        return self.transformer_encoder(tokens)

# DeepFake Detection Model
class DeepFakeDetector(nn.Module):
    def __init__(self):
        super(DeepFakeDetector, self).__init__()
        self.dense_feature_extractor = DenseNetFeatureExtractor()
        self.cross_vit = CrossViT(input_dim=1024)
        self.classifier = nn.Linear(1024, 2)

    def forward(self, x):
        features = self.dense_feature_extractor(x)
        features = features.unsqueeze(1)  # Shape: (batch_size, 1, 1024)
        encoded_features = self.cross_vit(features)
        output = self.classifier(encoded_features[:, 0, :])
        return output

# Load model and weights
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DeepFakeDetector().to(device)

model_path = "/content/deepfake_model.pth"
if os.path.exists(model_path):
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint, strict=False)
    model.eval()
    print("✅ Model loaded successfully!")
else:
    print(f"❌ Model file not found at {model_path}")
    exit()

# Load dataset
test_dataset = TestDataset(root_dir=extracted_test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Model evaluation

import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from torchvision import datasets, transforms


# Define device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Directory to save feature images
feature_output_dir = "/content/densenet_features1"
os.makedirs(feature_output_dir, exist_ok=True)

# Path to test dataset (modify if needed)
#test_dir = "/content/Dataset_f/Test"
test_dir=extracted_test_dir
# Define transformations
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load test dataset
test_dataset = datasets.ImageFolder(root=test_dir, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load trained model (ensure the model is defined and loaded properly)
model = torch.load("deepfake_model.pth", map_location=device)
model.eval()

# Model evaluation with F1-score calculation and feature extraction
correct = 0
total = 0
all_labels = []
all_predictions = []

with torch.no_grad():
    for batch_idx, (images, labels) in enumerate(test_loader):
        images, labels = images.to(device), labels.to(device)
        features = model.dense_feature_extractor(images)  # Extract DenseNet features

        # Save each feature map as an image
        for i in range(features.shape[0]):  # Iterate over batch
            feature_map = features[i].cpu().numpy()  # Convert to NumPy
            feature_map = np.mean(feature_map, axis=0)  # Average over channels

            # Normalize to 0-1 range
            feature_map = (feature_map - feature_map.min()) / (feature_map.max() - feature_map.min() + 1e-8)

            # Save feature image
            plt.imsave(os.path.join(feature_output_dir, f"feature_{batch_idx}_{i}.png"), feature_map, cmap='viridis')

        outputs = model(images)
        predicted = torch.argmax(outputs, dim=1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Store predictions and labels for F1-score calculation
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

if total > 0:
    accuracy = 100 * correct / total
    f1 = f1_score(all_labels, all_predictions, average='weighted')  # Weighted F1-score

    print(f" Model Test Accuracy: {accuracy:.2f}%")
    print(f" Model F1-Score: {f1:.4f}")
    print(f"Feature maps saved in: {feature_output_dir}")
else:
    print(" No data to evaluate.")




🔄 Extracting testdata.zip...
✅ Extraction complete!
✅ Model loaded successfully!
✅ Found 900 images in the dataset.


AttributeError: 'collections.OrderedDict' object has no attribute 'eval'

In [ ]:
import torch
import os
import zipfile
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms, models, datasets
from torch.utils.data import DataLoader, Dataset
from torch import nn
from sklearn.metrics import f1_score

# Ensure dataset is extracted before use
zip_path = "/content/Dataset_f.zip"
extract_to = "/content/Dataset_f"

if os.path.exists(zip_path) and zipfile.is_zipfile(zip_path):
    print("\U0001F504 Extracting Dataset_f.zip...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print("✅ Extraction complete!")

# Update dataset directory after extraction
extracted_test_dir = extract_to

# Check dataset presence
if not os.path.exists(extracted_test_dir) or not os.listdir(extracted_test_dir):
    print(f"❌ No images found in {extracted_test_dir}. Please verify the dataset.")
    exit()

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Custom dataset class for loading images
class TestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.data = []

        for subdir, _, files in os.walk(root_dir):
            for img_name in files:
                if img_name.lower().endswith((".png", ".jpg", ".jpeg")):
                    img_path = os.path.join(subdir, img_name)
                    label = 0 if "real" in subdir.lower() or "real" in img_name.lower() else 1
                    self.data.append((img_path, label))

        print(f"✅ Found {len(self.data)} images in the dataset.")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

# Feature extractor using DenseNet
class DenseNetFeatureExtractor(nn.Module):
    def __init__(self):
        super(DenseNetFeatureExtractor, self).__init__()
        densenet = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT)
        self.features = densenet.features
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = self.flatten(x)
        return x

# Transformer-based classifier (CrossViT)
class CrossViT(nn.Module):
    def __init__(self, input_dim=1024, hidden_dim=512, num_heads=8, num_layers=2):
        super(CrossViT, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=input_dim, nhead=num_heads, dim_feedforward=hidden_dim, batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)

    def forward(self, tokens):
        return self.transformer_encoder(tokens)

# DeepFake Detection Model
class DeepFakeDetector(nn.Module):
    def __init__(self):
        super(DeepFakeDetector, self).__init__()
        self.dense_feature_extractor = DenseNetFeatureExtractor()
        self.cross_vit = CrossViT(input_dim=1024)
        self.classifier = nn.Linear(1024, 2)

    def forward(self, x):
        features = self.dense_feature_extractor(x)
        features = features.unsqueeze(1)
        encoded_features = self.cross_vit(features)
        output = self.classifier(encoded_features[:, 0, :])
        return output

# Load model and weights
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DeepFakeDetector().to(device)

model_path = "/content/deepfake_model.pth"
if os.path.exists(model_path):
    checkpoint = torch.load(model_path, map_location=device)
    missing_keys, unexpected_keys = model.load_state_dict(checkpoint, strict=False)
    print("✅ Model loaded successfully!")
    print(f"Missing keys: {missing_keys}")
    print(f"Unexpected keys: {unexpected_keys}")
    model.eval()
else:
    print(f"❌ Model file not found at {model_path}")
    exit()


🔄 Extracting Dataset_f.zip...
✅ Extraction complete!
✅ Model loaded successfully!
Missing keys: []
Unexpected keys: ['dense_feature_extractor.fc.weight', 'dense_feature_extractor.fc.bias', 'cross_vit.transformer_encoder.layers.2.self_attn.in_proj_weight', 'cross_vit.transformer_encoder.layers.2.self_attn.in_proj_bias', 'cross_vit.transformer_encoder.layers.2.self_attn.out_proj.weight', 'cross_vit.transformer_encoder.layers.2.self_attn.out_proj.bias', 'cross_vit.transformer_encoder.layers.2.linear1.weight', 'cross_vit.transformer_encoder.layers.2.linear1.bias', 'cross_vit.transformer_encoder.layers.2.linear2.weight', 'cross_vit.transformer_encoder.layers.2.linear2.bias', 'cross_vit.transformer_encoder.layers.2.norm1.weight', 'cross_vit.transformer_encoder.layers.2.norm1.bias', 'cross_vit.transformer_encoder.layers.2.norm2.weight', 'cross_vit.transformer_encoder.layers.2.norm2.bias']


In [ ]:
import torch
import os
import zipfile
from PIL import Image
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from torch import nn
from torchvision import models

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Ensure dataset is extracted before use
zip_path = "/content/Dataset_f.zip"
extract_to = "/content/Test"

if os.path.exists(zip_path) and zipfile.is_zipfile(zip_path):
    print("🔄 Extracting testdata.zip...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print("✅ Extraction complete!")

# Update dataset directory after extraction
extracted_test_dir = extract_to

# Check dataset presence
if not os.path.exists(extracted_test_dir) or not os.listdir(extracted_test_dir):
    print(f"❌ No images found in {extracted_test_dir}. Please verify the dataset.")
    exit()

# Dataset class for loading images from subdirectories
class TestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.data = []

        # Scan subdirectories
        for subdir, _, files in os.walk(root_dir):
            for img_name in files:
                if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img_path = os.path.join(subdir, img_name)

                    # Assign label based on subdirectory or filename
                    label = 0 if "real" in subdir.lower() or "real" in img_name.lower() else 1
                    self.data.append((img_path, label))

        print(f"✅ Found {len(self.data)*3} images in the dataset.")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

# Feature extractor using DenseNet
class DenseNetFeatureExtractor(nn.Module):
    def __init__(self):
        super(DenseNetFeatureExtractor, self).__init__()
        densenet = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT)
        self.features = densenet.features
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = self.flatten(x)  # Shape: (batch_size, 1024)
        return x

# Transformer-based classifier (CrossViT)
class CrossViT(nn.Module):
    def __init__(self, input_dim=1024, hidden_dim=512, num_heads=8, num_layers=2):
        super(CrossViT, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=input_dim, nhead=num_heads, dim_feedforward=hidden_dim, batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)

    def forward(self, tokens):
        return self.transformer_encoder(tokens)

# DeepFake Detection Model
class DeepFakeDetector(nn.Module):
    def __init__(self):
        super(DeepFakeDetector, self).__init__()
        self.dense_feature_extractor = DenseNetFeatureExtractor()
        self.cross_vit = CrossViT(input_dim=1024)
        self.classifier = nn.Linear(1024, 2)

    def forward(self, x):
        features = self.dense_feature_extractor(x)
        features = features.unsqueeze(1)  # Shape: (batch_size, 1, 1024)
        encoded_features = self.cross_vit(features)
        output = self.classifier(encoded_features[:, 0, :])
        return output

# Load model and weights
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DeepFakeDetector().to(device)

model_path = "/content/deepfake_model.pth"
if os.path.exists(model_path):
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint, strict=False)
    model.eval()
    print("✅ Model loaded successfully!")
else:
    print(f"❌ Model file not found at {model_path}")
    exit()

# Load dataset
test_dataset = TestDataset(root_dir=extracted_test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)
'''
# Model evaluation
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        predicted = torch.argmax(outputs, dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

if total > 0:
    accuracy = 100 * correct / total
    print(f"✅ Model Test Accuracy: {accuracy:.2f}%")
else:
    print("❌ No data to evaluate.")
'''

🔄 Extracting testdata.zip...
✅ Extraction complete!
✅ Model loaded successfully!
✅ Found 900 images in the dataset.


'\n# Model evaluation\ncorrect = 0\ntotal = 0\n\nwith torch.no_grad():\n    for images, labels in test_loader:\n        images, labels = images.to(device), labels.to(device)\n        outputs = model(images)\n        predicted = torch.argmax(outputs, dim=1)\n        total += labels.size(0)\n        correct += (predicted == labels).sum().item()\n\nif total > 0:\n    accuracy = 100 * correct / total\n    print(f"✅ Model Test Accuracy: {accuracy:.2f}%")\nelse:\n    print("❌ No data to evaluate.")\n'